In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from nameparser import HumanName
import requests

# 2023 - Present

In [33]:
team = 'DET'

_source = {
    2023: 'https://web.archive.org/web/20230306062049/https://www.mlb.com/tigers/team/front-office'
}

In [62]:
df = pd.DataFrame()

for year in _source.keys():
    res = requests.get(_source[year])
    soup = BeautifulSoup(res.content)
    deps = [x.text.strip() for x in soup.select('h4')]

    temp_df = pd.DataFrame({
        'Name': [x.text.replace(' – ', ' - ').split('- ')[-1] for x in soup.select('p, h4')],
        'Title': [' - '.join(x.text.replace(' – ', ' - ').split('- ')[:-1]).strip() for x in soup.select('p, h4')]
    })
    
    temp_df['Name'] = temp_df.Name.str.split(', ')
    temp_df = temp_df.explode('Name')

    temp_df['NameLast'] = temp_df.Name.apply(lambda x: HumanName(x).last)
    temp_df['NameFirst'] = temp_df.Name.apply(lambda x: HumanName(x).first)

    temp_df['Department'] = temp_df.Name.apply(lambda x: x if x in deps else None)
    temp_df['Department'].ffill(inplace=True)

    temp_df['Year'] = year
    temp_df['Team'] = team
    temp_df['Source of Information'] = _source[year]
    
    temp_df = temp_df[temp_df.Title.str.len() > 0]
    temp_df = temp_df[temp_df.NameLast != '']

    df = pd.concat([
        df,
        # temp_df
        temp_df[['Year', 'Team', 'NameLast', 'NameFirst', 'Title', 'Department', 'Source of Information']]
    ])

In [65]:
df[['Year', 'Team', 'NameLast', 'NameFirst', 'Title', 'Department', 'Source of Information']] \
    .to_excel(f'../out/{team}-{min(_source.keys())}-to-{max(_source.keys())}.xlsx', index=False)